本周只有一个代码实践题

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
# from torch.utils.data import DataLoader,Dataset
import os
if not os.path.exists("DialoGPT"):
 !git clone https://github.com/microsoft/DialoGPT.git 

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
# model.train()
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

## k sampling

In [45]:
def top_p_filtering(logits, top_p = 0.9, filter_value = -float('Inf')): # nucleus filtering
    assert logits.dim() == 1
    sorted_logits, sorted_indices = torch.sort(logits, descending = True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim = -1), dim = -1)

    sorted_indices_to_remove = cumulative_probs > top_p
    
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value
    return logits 



In [46]:
def top_n_filtering(logits, top_n = 10, filter_value = -float('Inf')):
    assert logits.dim() == 1
    sorted_logits, sorted_indices = torch.sort(logits, descending = True)

    sorted_indices_to_remove = sorted_indices >= top_n
    sorted_indices_to_remove[..., :top_n] = 0
    sorted_indices_to_remove[..., top_n:] = 1

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value
    return logits 



In [48]:
def recalc():
    global conditioned_tokens
    global generated_tokens
    
    indexed_tokens = conditioned_tokens + generated_tokens

    tokens_tensor = torch.tensor([indexed_tokens]) # 
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    logits = predictions[0, -1, :]

    filtered_logits = top_n_filtering(logits)

    probabilities = F.softmax(filtered_logits, dim = -1)

    next_token = torch.multinomial(probabilities, 1)

    generated_tokens.append(next_token.item())
    return next_token.item() 

In [43]:
def generate():
    global conditioned_tokens
    global generated_tokens
    
    if len(tokenizer.decode(conditioned_tokens)) > 320:
        dc = tokenizer.decode(conditioned_tokens)
        dc = dc[len(dc) - 320:]
        idx = dc.find('<|endoftext|>')
        if idx != -1:
            dc = dc[idx + len('<|endoftext|>'):]
            
        conditioned_tokens = tokenizer.encode(dc)
        
    while True:
        result = recalc()
        
        if result == 50256:
            # end of text: 50256
            
            decoded_reply = tokenizer.decode(generated_tokens)
            
            to_print = decoded_reply
            if to_print.endswith('<|endoftext|>'):
                to_print = to_print[:-len('<|endoftext|>')]
            print(to_print)
            
            conditioned_tokens += (tokenizer.encode(decoded_reply))
            
            generated_tokens = []
            
            break 

## beam search

In [64]:
def recalc_beam_search(n_beam = 10):
    global conditioned_tokens
    global generated_tokens
    
    indexed_tokens = generated_tokens #conditioned_tokens + generated_tokens

    next_token = dict()
    for _tokens in indexed_tokens:
        tokens_tensor = torch.tensor([_tokens]) # 
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        logits = predictions[0, -1, :]
    
        probabilities = F.softmax(logits, dim = -1)

        sorted_probabilities, sorted_indices = torch.sort(probabilities, descending = True)
        for i in sorted_indices[:n_beam]: #probabilities.shape[1]:
            next_token[tuple(_tokens + [i.item()])] = probabilities[..., i,None]

    next_token2 = sorted(next_token.items(), key = lambda x: x[1], reverse=True)
    next_token2 = next_token2[:n_beam] 

    generated_tokens = [list(x[0]) for x in next_token2]
    return [x[0][-1] for x in next_token2] # return end of text 

In [68]:
def generate_beam_search():
    global conditioned_tokens
    global generated_tokens
    
    if len(tokenizer.decode(conditioned_tokens[0])) > 320:
        dc = tokenizer.decode(conditioned_tokens[0])
        dc = dc[len(dc) - 320:]
        idx = dc.find('<|endoftext|>')
        if idx != -1:
            dc = dc[idx + len('<|endoftext|>'):]
            
        conditioned_tokens = [tokenizer.encode(dc)]
    
    generated_tokens = conditioned_tokens

    while True:
        result = recalc_beam_search()
        if 50256 in result:
            # end of text: 50256
            
            decoded_reply = tokenizer.decode(generated_tokens[0])
            
            to_print = decoded_reply
            if to_print.endswith('<|endoftext|>'):
                to_print = to_print[:-len('<|endoftext|>')]
            
            print_start = to_print.find('<|endoftext|>') + len('<|endoftext|>')
            to_print = to_print[print_start:]
            print(to_print)
            
            conditioned_tokens += [tokenizer.encode(to_print)]
            
            generated_tokens = [] 
            
            break 

In [72]:
conditioned_tokens = []
generated_tokens = []
## k-sampling
first_text = "What is the meaning of life?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

## beam search 
conditioned_tokens = []
generated_tokens = []
conditioned_tokens += [tokenizer.encode('\t' + first_text) + [50256]]
print('Beam search:\n')
generate_beam_search()
print('#################\n')

K-sampling:

What's your favorite thing about life?
#################

Beam search:

The meaning of life
#################



In [73]:
conditioned_tokens = []
generated_tokens = []

first_text = "Does money buy happiness?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

## beam search 
conditioned_tokens = []
generated_tokens = []
conditioned_tokens += [tokenizer.encode('\t' + first_text) + [50256]]
print('Beam search:\n')
generate_beam_search()
print('#################\n')

K-sampling:

Money makes you happy
#################

Beam search:

If money buys happiness
#################



In [74]:
conditioned_tokens = []
generated_tokens = []

first_text = "what is the meaning of a good life?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

## beam search 
conditioned_tokens = []
generated_tokens = []
conditioned_tokens += [tokenizer.encode('\t' + first_text) + [50256]]
print('Beam search:\n')
generate_beam_search()
print('#################\n')

K-sampling:

So, how are you coping with life in the end, exactly?
#################

Beam search:

What is the meaning
#################



In [75]:
conditioned_tokens = []
generated_tokens = []

first_text = "How to be a good person?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

## beam search 
conditioned_tokens = []
generated_tokens = []
conditioned_tokens += [tokenizer.encode('\t' + first_text) + [50256]]
print('Beam search:\n')
generate_beam_search()
print('#################\n')

K-sampling:

how to be a good person
#################

Beam search:

what's the book?
#################

